# Model API

In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

# Parameters

In [3]:
num_classes = 10 # total classes (0-9 digits).
num_features = 32*32*3 # data features 
# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 256
display_step = 100
# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

# Importing Dataset

In [4]:
from keras.datasets import cifar10
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
print('shape x:\t',xtrain.shape)
print('shape y:\t',ytrain.shape)

170498071/170498071 [==============================] - 431s 3us/step
shape x:	 (50000, 32, 32, 3)
shape y:	 (50000, 1)


In [5]:
#Flatten Images to 1D vector of 784 features(28*28)
xtrain,xtest=xtrain.reshape([-1,num_features]),xtest.reshape([-1,num_features])

#Normalize images value from [0,255] to [0,1]
xtrain,xtest=xtrain/255.,xtest/255.

In [6]:
xtest[0].max()

1.0

In [7]:
train_data=tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


# Build the model

In [8]:
# Create TF Model.
class NeuralNet(Model):
     # Set layers.
     def __init__(self):
         super(NeuralNet, self).__init__()
         # First fully-connected hidden layer.
         self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
         # First fully-connected hidden layer.
         self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
         # Second fully-connecter hidden layer.
         self.out = layers.Dense(num_classes, activation=tf.nn.softmax)
     
    # Set forward pass.
     def call(self, x, is_training=False):
         x = self.fc1(x)
         x = self.fc2(x)
         x = self.out(x)
         if not is_training:
             # tf cross entropy expect logits without softmax, so only
             # apply softmax when not training.
             x = tf.nn.softmax(x)
         return x
# Build neural network model.
neural_net = NeuralNet()

In [9]:
def cross_entropy_loss(x, y):
    # Convert labels to int64 for tf loss
    y = tf.squeeze(y)  # Remove the extra dimension
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)


In [10]:
def accuracy(ypred, ytrue):
    correct_prediction = tf.equal(tf.argmax(ypred, 1), tf.cast(ytrue, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [11]:
#optimizer
optimizer=tf.optimizers.SGD(learning_rate)

In [12]:
# Optimization process.
def run_optimization(x, y):
     # Wrap computation inside a GradientTape for automatic differentiation
     with tf.GradientTape() as g:
         # Forward pass.
         pred = neural_net(x, is_training=True)
         # Compute loss.
         loss = cross_entropy_loss(pred, y)
        
     # Variables to update, i.e. trainable variables.
     trainable_variables = neural_net.trainable_variables
        
     # Compute gradients.
     gradients = g.gradient(loss, trainable_variables)
        
     # Update W and b following gradients.
     optimizer.apply_gradients(zip(gradients, trainable_variables))

In [13]:
#  Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps),1): 
     # Run the optimization to update W and b values.
     run_optimization(batch_x, batch_y)
                                          
     if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.248803, accuracy: 0.107788
step: 200, loss: 2.192740, accuracy: 0.112381
step: 300, loss: 2.159138, accuracy: 0.106644
step: 400, loss: 2.146268, accuracy: 0.104630
step: 500, loss: 2.138933, accuracy: 0.104919
step: 600, loss: 2.094764, accuracy: 0.107697
step: 700, loss: 2.100277, accuracy: 0.102966
step: 800, loss: 2.092839, accuracy: 0.106186
step: 900, loss: 2.069626, accuracy: 0.105225
step: 1000, loss: 2.079847, accuracy: 0.107452
step: 1100, loss: 2.076299, accuracy: 0.103409
step: 1200, loss: 2.040287, accuracy: 0.103622
step: 1300, loss: 2.114758, accuracy: 0.096146
step: 1400, loss: 2.085408, accuracy: 0.102280
step: 1500, loss: 2.071772, accuracy: 0.105362
step: 1600, loss: 2.064333, accuracy: 0.100876
step: 1700, loss: 2.046067, accuracy: 0.103714
step: 1800, loss: 2.058167, accuracy: 0.100174
step: 1900, loss: 2.064751, accuracy: 0.102386
step: 2000, loss: 2.052171, accuracy: 0.110062


# Accuracy Measure

In [14]:
pred = neural_net(xtest, is_training=False)
print("Test Accuracy: %f" % accuracy(pred, ytest))


Test Accuracy: 0.100000
